In [87]:
import time
print(time.ctime(time.time()))

Mon Dec  8 04:27:40 2025


# Advent of Code Day 8

Puzzle text available at:
https://adventofcode.com/2025/day/8

In [88]:
import os
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [89]:
# Day of calendar
day = 8
# Input filename
puzzle_test = 'input_day%02d_test.txt' %(day)
puzzle_input = 'input_day%02d.txt' %(day)

In [101]:
with open(puzzle_input, 'r') as f:
    coords = np.loadtxt(f, delimiter=',', dtype=int)
print(coords)

connections_limit = 1000

[[31316 90057 44547]
 [31517 64067  5099]
 [83231 82688 97747]
 ...
 [60770 54844 69309]
 [17340 58731 16307]
 [38583 44904 38383]]


## Part 1

Find the closest junction boxes and connect them

In [102]:
# Calculate pairwise distances between boxes using Scipy pdist
# Symmetric distance matrix
distance = squareform(pdist(coords))

# Set diagonal and lower triangle to NaN for visual purposes and avoid duplicate elements
distance[np.tril_indices_from(distance)] = np.nan

# Get upper triangle indices (excluding diagonal: k=1)
i_upper, j_upper = np.triu_indices(distance.shape[0], k=1)

# Get distances for upper triangle
upper_distances = distance[i_upper, j_upper]

# Sort by distance
sorted_idx = np.argsort(upper_distances)

# Get sorted (i,j) pairs and distances
sorted_pairs = list(zip(i_upper[sorted_idx], j_upper[sorted_idx]))

In [103]:
circuits = []

for i, pair in enumerate(sorted_pairs[:connections_limit]):
    # Solve manually first case
    if i == 0:   
        circuits.append(list(pair))
    else:
        # Find which circuits contain elements from pair
        matching_circuits = []
        for idx, connection in enumerate(circuits):
            if set(pair) & set(connection):
                matching_circuits.append(idx)
        
        if len(matching_circuits) == 0:
            # No overlap - create new circuit
            circuits.append(list(pair))
        elif len(matching_circuits) == 1:
            # One overlap - add box to that circuit
            common = set(pair) & set(circuits[matching_circuits[0]])
            if len(common) == 1:  # Avoid cycles
                circuits[matching_circuits[0]].extend(set(pair) - set(circuits[matching_circuits[0]]))
        else:
            # Multiple overlaps - MERGE circuits
            # Combine all matching circuits plus new pair
            merged = list(set(pair))
            for idx in sorted(matching_circuits, reverse=True):
                merged.extend(circuits[idx])
                circuits.pop(idx)  # Remove old circuit
            circuits.append(list(set(merged)))  # Add merged circuit (remove duplicates)

In [104]:
length_circuits = sorted([len(circuit) for circuit in circuits], reverse=True)
result_part1 = np.prod(length_circuits[:3])

In [105]:
print("what do you get if you multiply together the sizes of the three largest circuits?")
print(result_part1)

what do you get if you multiply together the sizes of the three largest circuits?
175500


## Part 2

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

The first part of the problem is the same - calculate the distances between the pair of boxes

Now the condition to finish the loop is to have all of the boxes in circuit

In [106]:
circuits = []

for i, pair in enumerate(sorted_pairs):
    # Solve manually first case
    if i == 0:   
        circuits.append(list(pair))
    else:
        # Find which circuits contain elements from pair
        matching_circuits = []
        for idx, connection in enumerate(circuits):
            if set(pair) & set(connection):
                matching_circuits.append(idx)
        
        if len(matching_circuits) == 0:
            # No overlap - create new circuit
            circuits.append(list(pair))
        elif len(matching_circuits) == 1:
            # One overlap - add box to that circuit
            common = set(pair) & set(circuits[matching_circuits[0]])
            if len(common) == 1:  # Avoid cycles
                circuits[matching_circuits[0]].extend(set(pair) - set(circuits[matching_circuits[0]]))
        else:
            # Multiple overlaps - MERGE circuits
            # Combine all matching circuits plus new pair
            merged = list(set(pair))
            for idx in sorted(matching_circuits, reverse=True):
                merged.extend(circuits[idx])
                circuits.pop(idx)  # Remove old circuit
            circuits.append(list(set(merged)))  # Add merged circuit (remove duplicates)
    
    # Part 2 addition
    if len(circuits) == 1 and len(circuits[0]) == len(coords):
        result_part2 = coords[pair[0],0] * coords[pair[1],0]
        break

In [107]:
print("What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?")
print(result_part2)

What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?
6934702555
